# Julia 機器學習：DecisionTree 決策樹

## 作業 030：乳癌預測資料集

請使用隨機森林模型建立一個分類模型，來預測乳癌資料集中，為良性或是惡性的腫瘤。

In [2]:
using DecisionTree, RDatasets, DataFrames, MLDataUtils, Statistics

## 讀取資料

In [3]:
biopsy = dataset("MASS", "biopsy")

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,Class
,String,Int32,Int32,Int32,Int32,Int32,Int32?,Int32,Int32,Int32,Cat…
1,1000025,5,1,1,1,2,1,3,1,1,benign
2,1002945,5,4,4,5,7,10,3,2,1,benign
3,1015425,3,1,1,1,2,2,3,1,1,benign
4,1016277,6,8,8,1,3,4,3,7,1,benign
5,1017023,4,1,1,3,2,1,3,1,1,benign
6,1017122,8,10,10,8,7,10,9,7,1,malignant
7,1018099,1,1,1,1,2,10,3,1,1,benign
8,1018561,2,1,2,1,2,1,3,1,1,benign
9,1033078,2,1,1,1,2,1,1,1,5,benign


In [11]:
dm_biopsy = dropmissing(biopsy)

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,Class
,String,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Cat…
1,1000025,5,1,1,1,2,1,3,1,1,benign
2,1002945,5,4,4,5,7,10,3,2,1,benign
3,1015425,3,1,1,1,2,2,3,1,1,benign
4,1016277,6,8,8,1,3,4,3,7,1,benign
5,1017023,4,1,1,3,2,1,3,1,1,benign
6,1017122,8,10,10,8,7,10,9,7,1,malignant
7,1018099,1,1,1,1,2,10,3,1,1,benign
8,1018561,2,1,2,1,2,1,3,1,1,benign
9,1033078,2,1,1,1,2,1,1,1,5,benign


In [50]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(dm_biopsy)))

683-element view(::Array{Int64,1}, [454, 482, 178, 417, 197, 167, 10, 625, 661, 470  …  11, 670, 673, 675, 666, 128, 220, 363, 372, 531]) with eltype Int64:
 454
 482
 178
 417
 197
 167
  10
 625
 661
 470
 144
 441
 260
   ⋮
 452
 273
  11
 670
 673
 675
 666
 128
 220
 363
 372
 531

In [53]:
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8)

([454, 482, 178, 417, 197, 167, 10, 625, 661, 470  …  580, 489, 67, 109, 131, 107, 386, 569, 278, 246], [243, 111, 176, 149, 355, 337, 160, 425, 602, 584  …  11, 670, 673, 675, 666, 128, 220, 363, 372, 531])

In [54]:
features=Matrix{Float64}(dm_biopsy[:,2:10])

683×9 Array{Float64,2}:
 5.0   1.0   1.0  1.0  2.0   1.0   3.0   1.0  1.0
 5.0   4.0   4.0  5.0  7.0  10.0   3.0   2.0  1.0
 3.0   1.0   1.0  1.0  2.0   2.0   3.0   1.0  1.0
 6.0   8.0   8.0  1.0  3.0   4.0   3.0   7.0  1.0
 4.0   1.0   1.0  3.0  2.0   1.0   3.0   1.0  1.0
 8.0  10.0  10.0  8.0  7.0  10.0   9.0   7.0  1.0
 1.0   1.0   1.0  1.0  2.0  10.0   3.0   1.0  1.0
 2.0   1.0   2.0  1.0  2.0   1.0   3.0   1.0  1.0
 2.0   1.0   1.0  1.0  2.0   1.0   1.0   1.0  5.0
 4.0   2.0   1.0  1.0  2.0   1.0   2.0   1.0  1.0
 1.0   1.0   1.0  1.0  1.0   1.0   3.0   1.0  1.0
 2.0   1.0   1.0  1.0  2.0   1.0   2.0   1.0  1.0
 5.0   3.0   3.0  3.0  2.0   3.0   4.0   4.0  1.0
 ⋮                           ⋮                
 3.0   1.0   1.0  1.0  2.0   1.0   2.0   3.0  1.0
 4.0   1.0   1.0  1.0  2.0   1.0   1.0   1.0  1.0
 1.0   1.0   1.0  1.0  2.0   1.0   1.0   1.0  8.0
 1.0   1.0   1.0  3.0  2.0   1.0   1.0   1.0  1.0
 5.0  10.0  10.0  5.0  4.0   5.0   4.0   4.0  1.0
 3.0   1.0   1.0  1.0  2.0   

In [55]:
labels = Vector(string.(dm_biopsy[:,11]))

683-element Array{String,1}:
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 ⋮
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "malignant"
 "malignant"

# DecisionTree

In [56]:
model = DecisionTree.DecisionTreeClassifier(max_depth=2)

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  nothing
root:                     nothing

In [57]:
DecisionTree.fit!(model, features, labels)

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  ["benign", "malignant"]
root:                     Decision Tree
Leaves: 4
Depth:  2

In [58]:
DecisionTree.print_tree(model, 5)

Feature 2, Threshold 2.5
L-> Feature 6, Threshold 3.5
    L-> benign : 393/395
    R-> benign : 13/23
R-> Feature 2, Threshold 4.5
    L-> malignant : 55/90
    R-> malignant : 172/175


In [68]:
new_b = [4.0,1.0,2.0,1.0,3.0,1.0,1.0,2.0,1.0]
ya = DecisionTree.predict(model, features[test_ind, :])

137-element Array{String,1}:
 "benign"
 "malignant"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 ⋮
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"

In [69]:
accuracy(ya, labels[test_ind])

0.9927007299270073

In [66]:
DecisionTree.predict_proba(model, new_b)

2-element Array{Float64,1}:
 0.992
 0.008

# RandomForest

In [60]:
model_2 = DecisionTree.RandomForestClassifier(n_trees=500, max_depth=9)

RandomForestClassifier
n_trees:             500
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           9
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             nothing
ensemble:            nothing

In [61]:
DecisionTree.fit!(model_2, features[train_ind, :], labels[train_ind])

RandomForestClassifier
n_trees:             500
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           9
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             ["benign", "malignant"]
ensemble:            Ensemble of Decision Trees
Trees:      500
Avg Leaves: 19.588
Avg Depth:  8.076

In [62]:
ŷ = DecisionTree.predict(model_2, features[test_ind, :])

137-element Array{String,1}:
 "benign"
 "malignant"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 ⋮
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"

In [63]:
accuracy(xs, ys) = mean(xs .== ys)

accuracy (generic function with 1 method)

In [64]:
accuracy(ŷ, labels[test_ind])

0.9927007299270073